# Summary
- The `EXISTING` sheet is assumed to contain the collected existing capacities collected prior to any reallocations intended to align with the PEMMDB. This sheet appears to be excluded from the modelling exercise, except for hydrogen-generating capacities.
- The `LAYER_POTENTIAL` sheet is viewed as containing the reallocated existing capacities (exclusing hydrogen-generating specific information) and the theoretical potentials per technology. Existing capacities are specified for both electricity- and hydrogen-generating offshore wind farms. Technology shares from `EXISTING` will be used to supplement the data.
- The `ZONE_POTENTIAL` sheet is considered as the source for achievable potentials for each node across all planning horizons. It establishes a nodal constraint on top of the theoretical potentials outlined by `LAYER_POTENTIAL`.
- **Existing capacities** will be read from the `LAYER_POTENTIAL` sheet, utilizing technology shares specified in `EXISTING` for hydrogen-generating capacities. A discrepancy of 526 MW for `DEOH002` in 2045 (across all scenarios) is noted when comparing existing capacities with `ZONE_POTENTIAL`. It remains uncertain which of the two values is correct: 5828.55 MW from `LAYER_POTENTIAL` or 6354.55 MW from `ZONE_POTENTIAL`.
- **Potentials** will be obtained from both the `LAYER_POTENTIAL` and the `ZONE_POTENTIAL` sheets. `LAYER_POTENTIAL` will establish a technology level constraint, while `ZONE_POTENTIAL` will restrict expansion across all technologies at each node. The same 526 MW discrepancy in `DEOH002` (across all planning horizons and scenarios) has been noted and needs to be addressed to ensure that existing capacities do not exceed their potential.

# Imports

In [214]:
import os
from pathlib import Path

import pandas as pd

In [215]:
if "notebooks" in os.getcwd():
    %cd ..

fn = Path("data/tyndp_2024_bundle/Offshore hubs/GENERATOR.xlsx")
tol = 1e-3

# Read data

Read data from the different sheets in `Offshore hubs/GENERATOR.xlsx`.

In [216]:
df_existing = pd.read_excel(fn, sheet_name="EXISTING").set_index(
    ["OFFSHORE_NODE", "OFFSHORE_NODE_TYPE", "YEAR", "SCENARIO", "TECHNOLOGY"]
)[["MW"]]
df_existing

MW
OFFSHORE_NODE OFFSHORE_NODE_TYPE YEAR SCENARIO           TECHNOLOGY         
BEIOH01       Hub                2030 Distributed Energy DC_FB_OH    3000.00
                                      Global Ambition    DC_FB_OH    3000.00
                                      National Trends    DC_FB_OH    3000.00
                                 2035 Distributed Energy DC_FB_OH    3000.00
                                      Global Ambition    DC_FB_OH    3000.00
...                                                                      ...
DEOR001       Radial             2045 National Trends    AC_FB_R      111.00
                                                         DC_FB_R     3549.75
                                 2050 Distributed Energy DC_FB_R     2699.75
                                      Global Ambition    DC_FB_R     2699.75
                                      National Trends    DC_FB_R     2699.75

[1371 rows x 1 columns]

In [217]:
df_layer = pd.read_excel(fn, sheet_name="LAYER_POTENTIAL").set_index(
    ["OFFSHORE_NODE", "OFFSHORE_NODE_TYPE", "YEAR", "SCENARIO", "TECH1", "TECH2"]
)[["EXISTING_MW", "MARGIN_MW", "MAX_MW"]]
df_layer

EXISTING_MW  \
OFFSHORE_NODE OFFSHORE_NODE_TYPE YEAR SCENARIO           TECH1   TECH2                
ALOR001       Radial             2030 Distributed Energy AC_FB_R NaN            0.0   
                                 2035 Distributed Energy AC_FB_R NaN            0.0   
                                 2040 Distributed Energy AC_FB_R NaN            0.0   
                                 2045 Distributed Energy AC_FB_R NaN            0.0   
                                 2050 Distributed Energy AC_FB_R NaN            0.0   
...                                                                             ...   
DEOR001       Radial             2030 National Trends    DC_FL_R NaN            0.0   
                                 2035 National Trends    DC_FL_R NaN            0.0   
                                 2040 National Trends    DC_FL_R NaN            0.0   
                                 2045 National Trends    DC_FL_R NaN            0.0   
                                 2050 National Trends    DC_FL_R NaN            0.0   

                                                                        MARGIN_MW  \
OFFSHORE_NODE OFFSHORE_NODE_TYPE YEAR SCENARIO           TECH1   TECH2              
ALOR001       Radial             2030 Distributed Energy AC_FB_R NaN          0.0   
                                 2035 Distributed Energy AC_FB_R NaN          0.0   
                                 2040 Distributed Energy AC_FB_R NaN          0.0   
                                 2045 Distributed Energy AC_FB_R NaN          0.0   
                                 2050 Distributed Energy AC_FB_R NaN          0.0   
...                                                                           ...   
DEOR001       Radial             2030 National Trends    DC_FL_R NaN          0.0   
                                 2035 National Trends    DC_FL_R NaN          0.0   
                                 2040 National Trends    DC_FL_R NaN          0.0   
                                 2045 National Trends    DC_FL_R NaN          0.0   
                                 2050 National Trends    DC_FL_R NaN          0.0   

                                                                        MAX_MW  
OFFSHORE_NODE OFFSHORE_NODE_TYPE YEAR SCENARIO           TECH1   TECH2          
ALOR001       Radial             2030 Distributed Energy AC_FB_R NaN       0.0  
                                 2035 Distributed Energy AC_FB_R NaN       0.0  
                                 2040 Distributed Energy AC_FB_R NaN       0.0  
                                 2045 Distributed Energy AC_FB_R NaN       0.0  
                                 2050 Distributed Energy AC_FB_R NaN       0.0  
...                                                                        ...  
DEOR001       Radial             2030 National Trends    DC_FL_R NaN       0.0  
                                 2035 National Trends    DC_FL_R NaN       0.0  
                                 2040 National Trends    DC_FL_R NaN       0.0  
                                 2045 National Trends    DC_FL_R NaN       0.0  
                                 2050 National Trends    DC_FL_R NaN       0.0  

[3180 rows x 3 columns]

In [218]:
df_zone = pd.read_excel(fn, sheet_name="ZONE_POTENTIAL").set_index(
    ["OFFSHORE_NODE", "OFFSHORE_NODE_TYPE", "YEAR", "SCENARIO"]
)[["EXISTING_MW", "MARGIN_MW", "MAX_MW"]]
df_zone

EXISTING_MW  \
OFFSHORE_NODE OFFSHORE_NODE_TYPE YEAR SCENARIO                          
ALOR001       Radial             2030 Distributed Energy         0.00   
                                 2035 Distributed Energy         0.00   
                                 2040 Distributed Energy         0.00   
                                 2045 Distributed Energy         0.00   
                                 2050 Distributed Energy         0.00   
...                                                               ...   
DEOR001       Radial             2030 National Trends         7548.75   
                                 2035 National Trends         7548.75   
                                 2040 National Trends         7486.75   
                                 2045 National Trends         3660.75   
                                 2050 National Trends         2699.75   

                                                          MARGIN_MW   MAX_MW  
OFFSHORE_NODE OFFSHORE_NODE_TYPE YEAR SCENARIO                                
ALOR001       Radial             2030 Distributed Energy        0.0     0.00  
                                 2035 Distributed Energy        0.0     0.00  
                                 2040 Distributed Energy        0.0     0.00  
                                 2045 Distributed Energy        0.0     0.00  
                                 2050 Distributed Energy        0.0     0.00  
...                                                             ...      ...  
DEOR001       Radial             2030 National Trends           0.0  7548.75  
                                 2035 National Trends           0.0  7548.75  
                                 2040 National Trends          62.0  7548.75  
                                 2045 National Trends        3888.0  7548.75  
                                 2050 National Trends        4849.0  7548.75  

[840 rows x 3 columns]

# Compare EXISTING and LAYER_POTENTIAL

This section evaluates the consistency of the  `EXISTING_MW` from the `EXISTING` and `LAYER_POTENTIAL` sheets. In `LAYER_POTENTIAL`, potential data are defined for both DC and H2 technologies together. For validation purposes, H2 technologies are omitted and the data is aggregated with the DC technologies.

In [219]:
df_existing_agg = (
    df_existing.reset_index(level=4)
    .replace("H2_", "DC_", regex=True)  # aggregates H2 technologies with DC
    .groupby(
        by=["OFFSHORE_NODE", "OFFSHORE_NODE_TYPE", "YEAR", "SCENARIO", "TECHNOLOGY"],
        dropna=False,
    )
    .sum()
)

In [220]:
df_layer_agg = (
    df_layer.rename_axis(index={"TECH1": "TECHNOLOGY"}).droplevel(
        "TECH2"
    )  # aggregates H2 technologies with DC
)

In [221]:
df_el_raw = df_existing_agg.join(
    (df_layer_agg.rename(columns={"EXISTING_MW": "MW"})[["MW"]]),
    how="outer",
    lsuffix="_E",
    rsuffix="_L",
)
df_el_raw

MW_E  \
OFFSHORE_NODE OFFSHORE_NODE_TYPE YEAR SCENARIO           TECHNOLOGY                 
ALOR001       Radial             2030 Distributed Energy AC_FB_R              NaN   
                                                         AC_FL_R              NaN   
                                                         DC_FB_R              NaN   
                                                         DC_FL_R              NaN   
                                      Global Ambition    AC_FB_R              NaN   
...                                                                           ...   
UKOR001       Radial             2050 Global Ambition    DC_FL_R     14962.282264   
                                      National Trends    AC_FB_R      8975.976664   
                                                         AC_FL_R        28.863405   
                                                         DC_FB_R     22023.495767   
                                                         DC_FL_R     14962.282264   

                                                                             MW_L  
OFFSHORE_NODE OFFSHORE_NODE_TYPE YEAR SCENARIO           TECHNOLOGY                
ALOR001       Radial             2030 Distributed Energy AC_FB_R         0.000000  
                                                         AC_FL_R         0.000000  
                                                         DC_FB_R         0.000000  
                                                         DC_FL_R         0.000000  
                                      Global Ambition    AC_FB_R         0.000000  
...                                                                           ...  
UKOR001       Radial             2050 Global Ambition    DC_FL_R     14962.282264  
                                      National Trends    AC_FB_R      8975.976664  
                                                         AC_FL_R        28.863405  
                                                         DC_FB_R     22023.495767  
                                                         DC_FL_R     14962.282264  

[3231 rows x 2 columns]

The methodology report indicates that reallocations are necessary to align with Radial capacities from the PEMMDB. Inconsistent radial capacities have been reassigned to offshore hubs for the purpose of comparing the two sheets.

In [222]:
df_el_raw_R = df_el_raw.query(
    "TECHNOLOGY.str.contains('_R') "  # only radial connexion
    "and MW_E != MW_L "  # when EXISTING and LAYER_POTENTIAL values are inconsistent
    "and ~(MW_E.isna() and MW_L == 0)"  # treat missing values and zeros as equivalent
)
df_el_raw_R

MW_E  \
OFFSHORE_NODE OFFSHORE_NODE_TYPE YEAR SCENARIO           TECHNOLOGY                 
BEOH001       Hub                2030 Distributed Energy DC_FB_R      2647.515945   
                                      Global Ambition    DC_FB_R      2647.515945   
                                      National Trends    DC_FB_R      2647.515945   
                                 2035 Distributed Energy DC_FB_R      3747.515945   
                                      Global Ambition    DC_FB_R      3747.515945   
                                      National Trends    DC_FB_R      3747.515945   
                                 2040 Distributed Energy DC_FB_R      4847.515945   
                                      Global Ambition    DC_FB_R      4847.515945   
                                      National Trends    DC_FB_R      4847.515945   
                                 2045 Distributed Energy DC_FB_R      4847.515945   
                                      Global Ambition    DC_FB_R      4847.515945   
                                      National Trends    DC_FB_R      4847.515945   
                                 2050 Distributed Energy DC_FB_R      4847.515945   
                                      Global Ambition    DC_FB_R      4847.515945   
                                      National Trends    DC_FB_R      4847.515945   
DEOH001       FarShoreHub        2030 Distributed Energy DC_FB_R      8868.000000   
                                      Global Ambition    DC_FB_R      8868.000000   
                                      National Trends    DC_FB_R      8868.000000   
                                 2035 Distributed Energy DC_FB_R     10868.000000   
                                      Global Ambition    DC_FB_R     10868.000000   
                                      National Trends    DC_FB_R     10868.000000   
                                 2040 Distributed Energy DC_FB_R     10468.000000   
                                      Global Ambition    DC_FB_R     10468.000000   
                                      National Trends    DC_FB_R     10468.000000   
                                 2045 Distributed Energy DC_FB_R      9680.000000   
                                      Global Ambition    DC_FB_R      9680.000000   
                                      National Trends    DC_FB_R      9680.000000   
                                 2050 Distributed Energy DC_FB_R      8780.000000   
                                      Global Ambition    DC_FB_R      8780.000000   
                                      National Trends    DC_FB_R      8780.000000   
DKWOH01       FarShoreHub        2035 Distributed Energy DC_FB_R      1000.000000   
                                      Global Ambition    DC_FB_R      1000.000000   
                                      National Trends    DC_FB_R      1000.000000   
                                 2040 Distributed Energy DC_FB_R      3000.000000   
                                      Global Ambition    DC_FB_R      3000.000000   
                                      National Trends    DC_FB_R      3000.000000   
                                 2045 Distributed Energy DC_FB_R      3000.000000   
                                      Global Ambition    DC_FB_R      3000.000000   
                                      National Trends    DC_FB_R      3000.000000   
                                 2050 Distributed Energy DC_FB_R      3000.000000   
                                      Global Ambition    DC_FB_R      3000.000000   
                                      National Trends    DC_FB_R      3000.000000   
SEOH002       Hub                2040 Distributed Energy DC_FB_R      1399.438094   
                                      Global Ambition    DC_FB_R      1399.438094   
                                      National Trends    DC_FB_R      1399.438094   
                                 2045 Distributed Energy DC_FB_R      1399.438094 

In [223]:
df_el = df_el_raw.copy().reset_index(level=4)
df_el.loc[df_el_raw_R.index, "TECHNOLOGY"] = "DC_FB_OH"  # reallocation to Offshore Hubs
df_el = (
    df_el.groupby(
        by=["OFFSHORE_NODE", "OFFSHORE_NODE_TYPE", "YEAR", "SCENARIO", "TECHNOLOGY"],
        dropna=False,
    )
    .sum()
    .assign(MW_diff=lambda x: x.MW_E - x.MW_L)
)

## Check that we have the same set of indexes

In [224]:
df_el[df_el["MW_diff"].isna()]

,,,,,MW_E,MW_L,MW_diff
OFFSHORE_NODE,OFFSHORE_NODE_TYPE,YEAR,SCENARIO,TECHNOLOGY,,,


No inconsistencies were observed.

## Compare EXISTING_MW

In [225]:
df_el_diff = df_el.query("abs(MW_diff) > @tol").sort_values(by="MW_diff")[
    ["MW_E", "MW_L", "MW_diff"]
]
df_el_diff.sort_values(by=["OFFSHORE_NODE", "OFFSHORE_NODE_TYPE", "YEAR", "SCENARIO"])

MW_E  \
OFFSHORE_NODE OFFSHORE_NODE_TYPE YEAR SCENARIO           TECHNOLOGY            
DEOH001       FarShoreHub        2050 Distributed Energy DC_FB_OH    62780.0   
                                      Global Ambition    DC_FB_OH    62780.0   
                                      National Trends    DC_FB_OH    62780.0   

                                                                        MW_L  \
OFFSHORE_NODE OFFSHORE_NODE_TYPE YEAR SCENARIO           TECHNOLOGY            
DEOH001       FarShoreHub        2050 Distributed Energy DC_FB_OH    62868.0   
                                      Global Ambition    DC_FB_OH    62868.0   
                                      National Trends    DC_FB_OH    62868.0   

                                                                     MW_diff  
OFFSHORE_NODE OFFSHORE_NODE_TYPE YEAR SCENARIO           TECHNOLOGY           
DEOH001       FarShoreHub        2050 Distributed Energy DC_FB_OH      -88.0  
                                      Global Ambition    DC_FB_OH      -88.0  
                                      National Trends    DC_FB_OH      -88.0

The 88 MW in `DEOH001` is likely associated with the alignment to the PEMMDB.

This section indicates that `EXISTING` and `LAYER_POTENTIAL` show consistency but do not represent the same data set. As the PEMMDB data will be used for replicating the TYNDP 2024, using `LAYER_POTENTIAL` as a reference for defining existing capacities is a logical approach. The `EXISTING` sheet is presumed to contain the collected data, but it seems to be excluded from the modelling exercise. However, it includes important information regarding existing hydrogen-generating capacities.

# Compare LAYER_POTENTIAL and ZONE_POTENTIAL

This section evaluates the consistency of the  `EXISTING_MW` and `MAX_MW` columns from the `LAYER_POTENTIAL` and `ZONE_POTENTIAL` sheets. The `MARGIN_MW` is presumed to be defined as `MAX_MW - EXISTING_MW` and is therefore not included in this analysis. Since `ZONE_POTENTIAL` does not provide disaggregated data at a technology level, `LAYER_POTENTIAL` data will be aggregated for the purpose of this consistency assesment.

In [226]:
df_lz = (
    df_layer.groupby(
        by=["OFFSHORE_NODE", "OFFSHORE_NODE_TYPE", "YEAR", "SCENARIO"], dropna=False
    )  # omit technologies due to differing levels of granularity
    .sum()
    .join(df_zone, how="outer", lsuffix="_L", rsuffix="_Z")
    .assign(
        EXISTING_MW_diff=lambda x: x.EXISTING_MW_L - x.EXISTING_MW_Z,
        MARGIN_MW_diff=lambda x: x.MARGIN_MW_L - x.MARGIN_MW_Z,
        MAX_MW_diff=lambda x: x.MAX_MW_L - x.MAX_MW_Z,
    )
)
df_lz

EXISTING_MW_L  \
OFFSHORE_NODE OFFSHORE_NODE_TYPE YEAR SCENARIO                            
ALOR001       Radial             2030 Distributed Energy         0.0000   
                                      Global Ambition            0.0000   
                                      National Trends            0.0000   
                                 2035 Distributed Energy         0.0000   
                                      Global Ambition            0.0000   
...                                                                 ...   
UKOR001       Radial             2045 Global Ambition        45240.6181   
                                      National Trends        45240.6181   
                                 2050 Distributed Energy     45990.6181   
                                      Global Ambition        45990.6181   
                                      National Trends        45990.6181   

                                                          MARGIN_MW_L  \
OFFSHORE_NODE OFFSHORE_NODE_TYPE YEAR SCENARIO                          
ALOR001       Radial             2030 Distributed Energy      0.00000   
                                      Global Ambition         0.00000   
                                      National Trends         0.00000   
                                 2035 Distributed Energy      0.00000   
                                      Global Ambition         0.00000   
...                                                               ...   
UKOR001       Radial             2045 Global Ambition      2172.46668   
                                      National Trends      2172.46668   
                                 2050 Distributed Energy   1422.46668   
                                      Global Ambition      1422.46668   
                                      National Trends      1422.46668   

                                                             MAX_MW_L  \
OFFSHORE_NODE OFFSHORE_NODE_TYPE YEAR SCENARIO                          
ALOR001       Radial             2030 Distributed Energy      0.00000   
                                      Global Ambition         0.00000   
                                      National Trends         0.00000   
                                 2035 Distributed Energy      0.00000   
                                      Global Ambition         0.00000   
...                                                               ...   
UKOR001       Radial             2045 Global Ambition     47413.08478   
                                      National Trends     47413.08478   
                                 2050 Distributed Energy  47413.08478   
                                      Global Ambition     47413.08478   
                                      National Trends     47413.08478   

                                                          EXISTING_MW_Z  \
OFFSHORE_NODE OFFSHORE_NODE_TYPE YEAR SCENARIO                            
ALOR001       Radial             2030 Distributed Energy         0.0000   
                                      Global Ambition            0.0000   
                                      National Trends            0.0000   
                                 2035 Distributed Energy         0.0000   
                                      Global Ambition            0.0000   
...                                                                 ...   
UKOR001       Radial             2045 Global Ambition        45240.6181   
                                      National Trends        45240.6181   
                                 2050 Distributed Energy     45990.6181   
                                      Global Ambition        45990.6181   
                                      National Trends        45990.6181   

                                                          MARGIN_MW_Z  \
OFFSHORE_NODE OFFSHORE_NODE_TYPE YEAR SCENARIO                          
ALOR001       Radial             2030 Distributed Energy    

## Check that we have the same set of indexes

In [227]:
(
    df_lz[df_lz[["EXISTING_MW_diff", "MARGIN_MW_diff", "MAX_MW_diff"]].isna()].dropna(
        how="all", axis=0
    )
)

,,,,EXISTING_MW_L,MARGIN_MW_L,MAX_MW_L,EXISTING_MW_Z,MARGIN_MW_Z,MAX_MW_Z,EXISTING_MW_diff,MARGIN_MW_diff,MAX_MW_diff
OFFSHORE_NODE,OFFSHORE_NODE_TYPE,YEAR,SCENARIO,,,,,,,,,


No inconsistencies were observed.

## Compare EXISTING_MW

In [228]:
col_existing = [c for c in df_lz.columns if "EXISTING" in c]
df_lz_ex = df_lz[col_existing].query("abs(EXISTING_MW_diff) > @tol")
df_lz_ex

EXISTING_MW_L  \
OFFSHORE_NODE OFFSHORE_NODE_TYPE YEAR SCENARIO                            
DEOH002       Hub                2045 Distributed Energy        5828.55   
                                      Global Ambition           5828.55   
                                      National Trends           5828.55   

                                                          EXISTING_MW_Z  \
OFFSHORE_NODE OFFSHORE_NODE_TYPE YEAR SCENARIO                            
DEOH002       Hub                2045 Distributed Energy        6354.55   
                                      Global Ambition           6354.55   
                                      National Trends           6354.55   

                                                          EXISTING_MW_diff  
OFFSHORE_NODE OFFSHORE_NODE_TYPE YEAR SCENARIO                              
DEOH002       Hub                2045 Distributed Energy            -526.0  
                                      Global Ambition               -526.0  
                                      National Trends               -526.0

The two tables indicates that the **existing potential** in `DEOH002`varies between the `LAYER_POTENTIAL` and `ZONE_POTENTIAL` sheets for 2045 across all scenarios. The `ZONE_POTENTIAL` sheet presents a higher value, with `LAYER_POTENTIAL` missing 526 MW in `DEOH002` in 2045.

## Compare MAX_MW

In [229]:
col_max = [c for c in df_lz.columns if "MAX" in c]
df_lz_max = (
    df_lz[col_max]
    .sort_values(by="MAX_MW_diff", ascending=False)
    .query("abs(MAX_MW_diff) > @tol")
)
df_lz_max.head(10)

MAX_MW_L  \
OFFSHORE_NODE OFFSHORE_NODE_TYPE YEAR SCENARIO                           
FIOH001       Hub                2030 Distributed Energy  65000.000000   
                                      Global Ambition     65000.000000   
                                      National Trends     65000.000000   
DEOH001       FarShoreHub        2030 Distributed Energy  62868.000000   
FIOH001       Hub                2035 Distributed Energy  65000.000000   
DEOH001       FarShoreHub        2030 Global Ambition     62868.000000   
                                      National Trends     62868.000000   
FIOH001       Hub                2035 Global Ambition     65000.000000   
                                      National Trends     65000.000000   
FROH002       Hub                2030 Global Ambition     38809.913867   

                                                              MAX_MW_Z  \
OFFSHORE_NODE OFFSHORE_NODE_TYPE YEAR SCENARIO                           
FIOH001       Hub                2030 Distributed Energy   7000.000000   
                                      Global Ambition      7000.000000   
                                      National Trends      7000.000000   
DEOH001       FarShoreHub        2030 Distributed Energy  18868.000000   
FIOH001       Hub                2035 Distributed Energy  21000.000000   
DEOH001       FarShoreHub        2030 Global Ambition     18868.000000   
                                      National Trends     18868.000000   
FIOH001       Hub                2035 Global Ambition     21000.000000   
                                      National Trends     21000.000000   
FROH002       Hub                2030 Global Ambition      2028.742923   

                                                           MAX_MW_diff  
OFFSHORE_NODE OFFSHORE_NODE_TYPE YEAR SCENARIO                          
FIOH001       Hub                2030 Distributed Energy  58000.000000  
                                      Global Ambition     58000.000000  
                                      National Trends     58000.000000  
DEOH001       FarShoreHub        2030 Distributed Energy  44000.000000  
FIOH001       Hub                2035 Distributed Energy  44000.000000  
DEOH001       FarShoreHub        2030 Global Ambition     44000.000000  
                                      National Trends     44000.000000  
FIOH001       Hub                2035 Global Ambition     44000.000000  
                                      National Trends     44000.000000  
FROH002       Hub                2030 Global Ambition     36781.170944

In [230]:
df_lz_max.tail(10)

MAX_MW_L  \
OFFSHORE_NODE OFFSHORE_NODE_TYPE YEAR SCENARIO                           
NOSOR01       Radial             2045 Distributed Energy   2046.442403   
UKOH003       Hub                2045 National Trends     16914.263202   
                                      Distributed Energy  16914.263202   
                                      Global Ambition     16914.263202   
DEOH002       Hub                2045 Global Ambition      5828.550000   
                                 2050 National Trends      5828.550000   
                                      Global Ambition      5828.550000   
                                      Distributed Energy   5828.550000   
                                 2045 National Trends      5828.550000   
                                      Distributed Energy   5828.550000   

                                                              MAX_MW_Z  \
OFFSHORE_NODE OFFSHORE_NODE_TYPE YEAR SCENARIO                           
NOSOR01       Radial             2045 Distributed Energy   1909.712163   
UKOH003       Hub                2045 National Trends     16827.086690   
                                      Distributed Energy  16827.086690   
                                      Global Ambition     16827.086690   
DEOH002       Hub                2045 Global Ambition      6354.550000   
                                 2050 National Trends      6354.550000   
                                      Global Ambition      6354.550000   
                                      Distributed Energy   6354.550000   
                                 2045 National Trends      6354.550000   
                                      Distributed Energy   6354.550000   

                                                          MAX_MW_diff  
OFFSHORE_NODE OFFSHORE_NODE_TYPE YEAR SCENARIO                         
NOSOR01       Radial             2045 Distributed Energy   136.730240  
UKOH003       Hub                2045 National Trends       87.176512  
                                      Distributed Energy    87.176512  
                                      Global Ambition       87.176512  
DEOH002       Hub                2045 Global Ambition     -526.000000  
                                 2050 National Trends     -526.000000  
                                      Global Ambition     -526.000000  
                                      Distributed Energy  -526.000000  
                                 2045 National Trends     -526.000000  
                                      Distributed Energy  -526.000000

As can be seen from the two tables above, the `LAYER_POTENTIAL` and `ZONE_POTENTIAL` sheets show discrepancies in the **maximum potential** specified per node, year, and scenario.

The 526 MW deficit in `DEOH002` in `LAYER_POTENTIAL` corresponds to the 526 MW negative difference in existing capacities discovered above.

Understanding the positive differences is less straightforward. For instance, the 58 GW difference in `FIOH001` can be explained by examining all the planning horizons, as the `MAX_MW` of `ZONE_POTENTIAL` varies over time. To validate this observation, the maximum potential of `ZONE_POTENTIAL` is extracted and compared with the potential of the `LAYER_POTENTIAL`.

In [231]:
df_zone_max = (
    df_zone.groupby(by=["OFFSHORE_NODE", "OFFSHORE_NODE_TYPE", "SCENARIO"])
    .max()[["MAX_MW"]]
    .rename(columns={"MAX_MW": "MAX_ZONE_MW"})
)
(
    df_lz[col_max]
    .join(df_zone_max)
    .assign(
        MAX_ZONE_MW_diff=lambda x: x.MAX_MW_L - x.MAX_ZONE_MW,
    )
    .query("abs(MAX_ZONE_MW_diff) > @tol")
)

MAX_MW_L  MAX_MW_Z  \
OFFSHORE_NODE OFFSHORE_NODE_TYPE YEAR SCENARIO                                 
DEOH002       Hub                2030 Distributed Energy   5828.55   4104.55   
                                      Global Ambition      5828.55   4104.55   
                                      National Trends      5828.55   4104.55   
                                 2035 Distributed Energy   5828.55   4104.55   
                                      Global Ambition      5828.55   4104.55   
                                      National Trends      5828.55   4104.55   
                                 2040 Distributed Energy   5828.55   5104.55   
                                      Global Ambition      5828.55   5104.55   
                                      National Trends      5828.55   5104.55   
                                 2045 Distributed Energy   5828.55   6354.55   
                                      Global Ambition      5828.55   6354.55   
                                      National Trends      5828.55   6354.55   
                                 2050 Distributed Energy   5828.55   6354.55   
                                      Global Ambition      5828.55   6354.55   
                                      National Trends      5828.55   6354.55   

                                                          MAX_MW_diff  \
OFFSHORE_NODE OFFSHORE_NODE_TYPE YEAR SCENARIO                          
DEOH002       Hub                2030 Distributed Energy       1724.0   
                                      Global Ambition          1724.0   
                                      National Trends          1724.0   
                                 2035 Distributed Energy       1724.0   
                                      Global Ambition          1724.0   
                                      National Trends          1724.0   
                                 2040 Distributed Energy        724.0   
                                      Global Ambition           724.0   
                                      National Trends           724.0   
                                 2045 Distributed Energy       -526.0   
                                      Global Ambition          -526.0   
                                      National Trends          -526.0   
                                 2050 Distributed Energy       -526.0   
                                      Global Ambition          -526.0   
                                      National Trends          -526.0   

                                                          MAX_ZONE_MW  \
OFFSHORE_NODE OFFSHORE_NODE_TYPE YEAR SCENARIO                          
DEOH002       Hub                2030 Distributed Energy      6354.55   
                                      Global Ambition         6354.55   
                                      National Trends         6354.55   
                                 2035 Distributed Energy      6354.55   
                                      Global Ambition         6354.55   
                                      National Trends         6354.55   
                                 2040 Distributed Energy      6354.55   
                                      Global Ambition         6354.55   
                                      National Trends         6354.55   
                                 2045 Distributed Energy      6354.55   
                                      Global Ambition         6354.55   
                                      National Trends         6354.55   
                                 2050 Distributed Energy      6354.55   
                                      Global Ambition         6354.55   
                                      National Trends         6354.55   

                                                          MAX_ZONE_MW_diff  
OFFSHORE_NODE OFFSHORE_NODE_TYPE YEAR SCENARIO                              
DEOH002       Hub                2030 Distributed Energy   

The previous assumption has been confirmed. The maximum values of `ZONE_POTENTIAL` are equal to the values observed in `LAYER_POTENTIAL`.

The remaining issue with the data is the 526 MW deficit in `DEOH002`, which is assumed to be missing in the `LAYER_POTENTIAL` data (see above).

This section concludes that `LAYER_POTENTIAL` and `ZONE_POTENTIAL` show consistency but do not represent the same data set either. It may be relevant to consider evolving maximum potential, as technological advancements could impact the definition of  potentials as trajectories. The `MAX_MW` specified in the `LAYER_POTENTIAL` is regarded as the theoretical potentials, while the `MAX_MW` from the `ZONE_POTENTIAL` is considered as the achievable potential at a given planning horizon.

The existing capacities (`MW` from `LAYER_POTENTIAL`) align with their achievable potentials (`MAX_MW` from `ZONE_POTENTIAL`). To demonstrate it, we calculate the corrected potential `MAX_MW_corr` for `LAYER_POTENTIAL`, considering the planning horizon constraint from `ZONE_POTENTIAL`, and compare it with the related existing potential. This approach simplifies the analysis by not accounting for technology shares.

In [232]:
df_lz_corr = (
    df_lz.join(df_zone_max)
    .assign(MAX_MW_corr=lambda x: x.MAX_MW_L / x.MAX_ZONE_MW * x.MAX_MW_Z)
    .fillna(0)
    .assign(
        issue=lambda x: x.EXISTING_MW_L
        > x.MAX_MW_corr,  # check if current capacities exceed potential
        gap=lambda x: x.EXISTING_MW_L - x.MAX_MW_corr,
    )
    .query("gap > @tol")
    .sort_values(by="gap", ascending=False)
)
df_lz_corr[
    [
        "EXISTING_MW_L",
        "MAX_MW_L",
        "MAX_MW_Z",
        "MAX_ZONE_MW",
        "MAX_MW_corr",
        "issue",
        "gap",
    ]
]

EXISTING_MW_L  \
OFFSHORE_NODE OFFSHORE_NODE_TYPE YEAR SCENARIO                            
DEOH002       Hub                2040 Distributed Energy        5104.55   
                                      Global Ambition           5104.55   
                                      National Trends           5104.55   
                                 2030 Distributed Energy        4104.55   
                                      Global Ambition           4104.55   
                                      National Trends           4104.55   
                                 2035 Distributed Energy        4104.55   
                                      Global Ambition           4104.55   
                                      National Trends           4104.55   

                                                          MAX_MW_L  MAX_MW_Z  \
OFFSHORE_NODE OFFSHORE_NODE_TYPE YEAR SCENARIO                                 
DEOH002       Hub                2040 Distributed Energy   5828.55   5104.55   
                                      Global Ambition      5828.55   5104.55   
                                      National Trends      5828.55   5104.55   
                                 2030 Distributed Energy   5828.55   4104.55   
                                      Global Ambition      5828.55   4104.55   
                                      National Trends      5828.55   4104.55   
                                 2035 Distributed Energy   5828.55   4104.55   
                                      Global Ambition      5828.55   4104.55   
                                      National Trends      5828.55   4104.55   

                                                          MAX_ZONE_MW  \
OFFSHORE_NODE OFFSHORE_NODE_TYPE YEAR SCENARIO                          
DEOH002       Hub                2040 Distributed Energy      6354.55   
                                      Global Ambition         6354.55   
                                      National Trends         6354.55   
                                 2030 Distributed Energy      6354.55   
                                      Global Ambition         6354.55   
                                      National Trends         6354.55   
                                 2035 Distributed Energy      6354.55   
                                      Global Ambition         6354.55   
                                      National Trends         6354.55   

                                                          MAX_MW_corr  issue  \
OFFSHORE_NODE OFFSHORE_NODE_TYPE YEAR SCENARIO                                 
DEOH002       Hub                2040 Distributed Energy  4682.019168   True   
                                      Global Ambition     4682.019168   True   
                                      National Trends     4682.019168   True   
                                 2030 Distributed Energy  3764.794502   True   
                                      Global Ambition     3764.794502   True   
                                      National Trends     3764.794502   True   
                                 2035 Distributed Energy  3764.794502   True   
                                      Global Ambition     3764.794502   True   
                                      National Trends     3764.794502   True   

                                                                 gap  
OFFSHORE_NODE OFFSHORE_NODE_TYPE YEAR SCENARIO                        
DEOH002       Hub                2040 Distributed Energy  422.530832  
                                      Global Ambition     422.530832  
                                      National Trends     422.530832  
                                 2030 Distributed Energy  339.755498  
                                      Global Ambition     339.755498  
                                      National Trends     339.755498  
                                 2035 Distributed Energy  339.755498  
                           

In this table, we demonstrate that the potentials, when considering the planning horizon, align with the existing capacities. Nonetheless, we already identified a deficit of 526 MW across all scenarios and planning horizons for `DEOH002` in `MAX_MW_L`. Addressing this deficit close the remaining gaps.

In [233]:
df_lz_corr_ = df_lz_corr.copy()

df_lz_corr_["MAX_MW_L"] = df_lz_corr_["MAX_MW_L"] + 526.0
df_lz_corr_ = (
    df_lz_corr_.assign(MAX_MW_corr=lambda x: x.MAX_MW_L / x.MAX_ZONE_MW * x.MAX_MW_Z)
    .fillna(0)
    .assign(
        issue=lambda x: x.EXISTING_MW_L
        > x.MAX_MW_corr,  # check if current capacities exceed potential
        gap=lambda x: x.EXISTING_MW_L - x.MAX_MW_corr,
    )
    .query("gap > @tol")
)

df_lz_corr_

,,,,EXISTING_MW_L,MARGIN_MW_L,MAX_MW_L,EXISTING_MW_Z,MARGIN_MW_Z,MAX_MW_Z,EXISTING_MW_diff,MARGIN_MW_diff,MAX_MW_diff,MAX_ZONE_MW,MAX_MW_corr,issue,gap
OFFSHORE_NODE,OFFSHORE_NODE_TYPE,YEAR,SCENARIO,,,,,,,,,,,,,


This confirms that the `MAX_MW` specified in `ZONE_POTENTIAL` represents achievable potentials at a specific planning horizon, in accordance with the `EXISTING_MW` from `LAYER_POTENTIAL` at any time.

Finally, we can demonstrate that some `EXISTING_MW` from `LAYER_POTENTIAL` exceed their corresponding `MAX_MW` from `ZONE_POTENTIAL`, when factoring in technology shares. For example, at node `EEOH001` in 2040, there is an existing `DC_FB_OH` capacity of 3.5 GW.

In [234]:
query_str = (
    "OFFSHORE_NODE=='EEOH001' and SCENARIO == 'Distributed Energy' and YEAR == 2040"
)
df_layer.query(query_str)

EXISTING_MW  \
OFFSHORE_NODE OFFSHORE_NODE_TYPE YEAR SCENARIO           TECH1    TECH2                   
EEOH001       Hub                2040 Distributed Energy AC_FB_R  NaN               0.0   
                                                         AC_FL_R  NaN               0.0   
                                                         DC_FB_OH H2_FB_OH       3500.0   
                                                         DC_FL_OH H2_FL_OH          0.0   

                                                                              MARGIN_MW  \
OFFSHORE_NODE OFFSHORE_NODE_TYPE YEAR SCENARIO           TECH1    TECH2                   
EEOH001       Hub                2040 Distributed Energy AC_FB_R  NaN       5246.125270   
                                                         AC_FL_R  NaN          0.000000   
                                                         DC_FB_OH H2_FB_OH  1252.224347   
                                                         DC_FL_OH H2_FL_OH     1.650382   

                                                                                 MAX_MW  
OFFSHORE_NODE OFFSHORE_NODE_TYPE YEAR SCENARIO           TECH1    TECH2                  
EEOH001       Hub                2040 Distributed Energy AC_FB_R  NaN       5246.125270  
                                                         AC_FL_R  NaN          0.000000  
                                                         DC_FB_OH H2_FB_OH  4752.224347  
                                                         DC_FL_OH H2_FL_OH     1.650382

The `MAX_MW` in `LAYER_POTENTIAL` is 10 GW.

In [235]:
df_layer.query(query_str).groupby(by="YEAR").sum()

,EXISTING_MW,MARGIN_MW,MAX_MW
YEAR,,,
2040,3500.0,6500.0,10000.0


In constrast, the `ZONE_POTENTIAL` reflects a `MAX_MW` of 7 GW.

In [236]:
df_zone.query(query_str)

,,,,EXISTING_MW,MARGIN_MW,MAX_MW
OFFSHORE_NODE,OFFSHORE_NODE_TYPE,YEAR,SCENARIO,,,
EEOH001,Hub,2040,Distributed Energy,3500.0,3500.0,7000.0


Applying the technology share to the zone potential does not align, as the `MAX_MW` indicates 3.3 GW while the `EXISTING_MW` is 3.5 GW.

In [237]:
df_layer.query(query_str).pipe(lambda df: df / df.sum() * df_zone.query(query_str))

EXISTING_MW  \
OFFSHORE_NODE OFFSHORE_NODE_TYPE YEAR SCENARIO           TECH1    TECH2                   
EEOH001       Hub                2040 Distributed Energy AC_FB_R  NaN               0.0   
                                                         AC_FL_R  NaN               0.0   
                                                         DC_FB_OH H2_FB_OH       3500.0   
                                                         DC_FL_OH H2_FL_OH          0.0   

                                                                              MARGIN_MW  \
OFFSHORE_NODE OFFSHORE_NODE_TYPE YEAR SCENARIO           TECH1    TECH2                   
EEOH001       Hub                2040 Distributed Energy AC_FB_R  NaN       2824.836684   
                                                         AC_FL_R  NaN          0.000000   
                                                         DC_FB_OH H2_FB_OH   674.274649   
                                                         DC_FL_OH H2_FL_OH     0.888667   

                                                                                 MAX_MW  
OFFSHORE_NODE OFFSHORE_NODE_TYPE YEAR SCENARIO           TECH1    TECH2                  
EEOH001       Hub                2040 Distributed Energy AC_FB_R  NaN       3672.287689  
                                                         AC_FL_R  NaN          0.000000  
                                                         DC_FB_OH H2_FB_OH  3326.557043  
                                                         DC_FL_OH H2_FL_OH     1.155268

This highlights that `ZONE_POTENTIAL` sets potentials at node levels, and this information should be added as a extra constraint to the model.